# import lib

In [ ]:
import pandas as pd
import pyodbc

# load csv file and create dataframe

In [ ]:
# Load CSV file
csv_file_path = r"D:\Workplace\code\ADY_prj\Used cars prediction\cars_dataframe_with_details.csv"
df = pd.read_csv(csv_file_path)


df_sorted = df.sort_values(by='Year')
print(df_sorted)

# handle missing values

In [ ]:
df2 = df_sorted.fillna(value='None')

# convert all value type to 'string'

In [ ]:
df2 = df2.astype(str)

# sql sever connection setup

In [ ]:
server = 'NITRO\\SQLEXPRESS' 
database = 'cars_data' 

driver = 'ODBC Driver 17 for SQL Server'

# Adjust connection string for Windows Authentication
connection_string = f'DRIVER={{{driver}}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

# create table in sql sever

In [ ]:
# create table cars_general_inf
create_table_query = '''
IF NOT EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'cars_general_inf') AND type in (N'U'))
BEGIN
    CREATE TABLE cars_general_inf (
        car_id INT PRIMARY KEY,
        link VARCHAR(255),
        title VARCHAR(255),
        manufacturer VARCHAR(255)      
    );
END;
'''

try:
    cursor.execute(create_table_query)
    conn.commit()
    print("Table 'cars_general_inf' created or already exists.")
except Exception as e:
    print(f"Error creating general table: {e}")


cars_brand = df['manufacturer']

# loop through each brand to create table ('manufacturer' in csv file)
def create_table_for_each_brand(cursor, cars_brand):
    for brand in cars_brand:
        table_name = f"[{brand.lower().replace(' ', '_').replace('-', '_')}]"  
        create_table = f'''
        IF NOT EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'{table_name}') AND type in (N'U'))
        BEGIN
            CREATE TABLE {table_name} (
                car_id INT REFERENCES cars_general_inf(car_id),
                title VARCHAR(255),
                year VARCHAR(255),
                Body_Type VARCHAR(50), 
                Origin VARCHAR(50),
                Mileage VARCHAR(255),
                City VARCHAR(50),
                District VARCHAR(50),
                Transmission VARCHAR(50),
                Fuel_Type VARCHAR(50),
                Price VARCHAR(255),
                sale_date VARCHAR(50),
                Manufacturer VARCHAR(255)
            );
        END;
        '''
        try:
            cursor.execute(create_table)
            conn.commit()
            print(f"Table '{table_name}' created or already exists.")
        except Exception as e:
            print(f"Error creating table '{table_name}': {e}")


create_table_for_each_brand(cursor, cars_brand)


# insert data from data frame into sql sever

In [ ]:
for index, row in df2.iterrows():
    # insert data to cars_general_inf
    insert_general_query = '''
    INSERT INTO cars_general_inf (car_id, link, title, manufacturer)
    VALUES (?, ?, ?, ?);
    '''
    try:
        cursor.execute(insert_general_query, 
                    row['car_id'],
                    row['Link'], 
                    row['Title'], 
                    row['manufacturer'])

    except Exception as e:
        print(f"Error inserting into 'cars_general_inf' for row {index}: {e}")


    # insert data to each brand table
    table_name = f"[{row['manufacturer'].lower().replace(' ', '_').replace('-', '_')}]"
    insert_brand_query = f'''
    INSERT INTO {table_name} (car_id, title, year, Body_Type, Origin, Mileage, City, District, Transmission, Fuel_Type, Price, sale_date, Manufacturer)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
    '''
    try:
        cursor.execute(insert_brand_query, 
                    row['car_id'],
                    row['Title'], 
                    row['Year'], 
                    row['Body Type'], 
                    row['Origin'], 
                    row['Mileage'], 
                    row['City'], 
                    row['District'], 
                    row['Transmission'], 
                    row['Fuel Type'], 
                    row['Price'], 
                    row['Sale Date'], 
                    row['manufacturer'])
    except Exception as e:
        print(f"Error at '{table_name}': {e}")

# commit
conn.commit()

# close connection

In [ ]:
# Close connection
cursor.close()
conn.close()

print("Data successfully inserted into SQL Server.")